In [0]:
# Understanding derivatives
# https://colab.research.google.com/
import os

In [0]:
os.listdir()

['.config', 'diabetes.csv.gz', 'sample_data']

In [0]:
import torch
import numpy as np
import matplotlib.pyplot as plt

In [0]:
import torch.nn as nn

In [0]:
import pandas as pd


In [0]:
df = pd.read_csv('diabetes.csv.gz', compression='gzip', header=None, dtype=np.float32)

In [0]:
df

,0,1,2,3,4,5,6,7,8
0,-0.294118,0.487437,0.180328,-0.292929,0.000000,0.001490,-0.531170,-0.033333,0.0
1,-0.882353,-0.145729,0.081967,-0.414141,0.000000,-0.207153,-0.766866,-0.666667,1.0
2,-0.058823,0.839196,0.049180,0.000000,0.000000,-0.305514,-0.492741,-0.633333,0.0
3,-0.882353,-0.105528,0.081967,-0.535354,-0.777778,-0.162444,-0.923997,0.000000,1.0
4,0.000000,0.376884,-0.344262,-0.292929,-0.602837,0.284650,0.887276,-0.600000,0.0
...,...,...,...,...,...,...,...,...,...
754,0.176471,0.015075,0.245902,-0.030303,-0.574468,-0.019374,-0.920581,0.400000,1.0
755,-0.764706,0.226131,0.147541,-0.454545,0.000000,0.096870,-0.776260,-0.800000,1.0
756,-0.411765,0.216080,0.180328,-0.535354,-0.735225,-0.219076,-0.857387,-0.700000,1.0
757,-0.882353,0.266332,-0.016393,0.000000,0.000000,-0.102832,-0.768574,-0.133333,0.0


In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,0:8],
                                                    df.iloc[:,8], random_state=8 )

In [0]:
X_train.shape, X_test.shape

((569, 8), (190, 8))

In [0]:
X_train = torch.from_numpy(X_train.values)
X_test = torch.from_numpy(X_test.values)
y_train = torch.from_numpy(y_train.values)
y_test = torch.from_numpy(y_test.values)

In [0]:
X_train.dtype

torch.float32

In [0]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(8,16)
        self.l2 = nn.Linear(16,22)
        self.l3 = nn.Linear(22,1)
    def forward(self,x):
        x1 = torch.relu(self.l1(x))
        x2 = torch.relu(self.l2(x1))
        x3 = torch.sigmoid(self.l3(x2))
        return x3



In [0]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(3,10)
        self.l2 = nn.Linear(10,1)
    def forward(self,x):
        return self.l1(x)



In [0]:
def accuracy(truth, preds):
    return (truth == preds).float().mean()

In [0]:
model = Model()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [0]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.2705,  0.0564,  0.2240, -0.1739, -0.2088,  0.1190, -0.3208, -0.0908],
         [ 0.1587,  0.1620,  0.1861, -0.0203,  0.0742,  0.3273, -0.3309, -0.1204],
         [-0.0736, -0.2303, -0.2425,  0.2312, -0.2762,  0.1173, -0.1089, -0.1654],
         [-0.2091,  0.3450, -0.1660, -0.1198, -0.3388, -0.0917,  0.3044,  0.0532],
         [ 0.2128,  0.3139, -0.0062,  0.0932,  0.2798,  0.1558, -0.2298, -0.2112],
         [ 0.2966,  0.1710,  0.1405,  0.1134, -0.1843, -0.1430, -0.3352, -0.2998],
         [-0.0251,  0.3137, -0.3382, -0.3471,  0.0951, -0.3513, -0.1995,  0.3220],
         [-0.0723,  0.1397, -0.0922, -0.0748,  0.1307,  0.1339, -0.0739,  0.3411],
         [-0.2699,  0.0661,  0.2806, -0.3526, -0.1465, -0.0147, -0.0545, -0.1002],
         [ 0.0404,  0.2012,  0.0016, -0.2944,  0.2509,  0.2755, -0.2330, -0.0783],
         [ 0.2613, -0.0157, -0.0548,  0.0924, -0.2571,  0.2294,  0.3347, -0.0603],
         [ 0.2093,  0.0045,  0.1050,  0.3089,  0.0981, -0.1156, 

In [0]:
for epoch in range(501):
    y_pred = model.forward(X_train)
    loss = criterion(y_pred, y_train)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    with torch.no_grad():
        pred_test = model.forward(X_test)
        pred_test = pred_test >0.5
        acc = accuracy(y_test, pred_test.view(y_test.shape))
        if epoch%50 ==0:
            print(epoch, acc)



/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:498: UserWarning: Using a target size (torch.Size([569])) that is different to the input size (torch.Size([569, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


0 tensor(0.3053)
50 tensor(0.6842)
100 tensor(0.7105)
150 tensor(0.7842)
200 tensor(0.7947)
250 tensor(0.8053)
300 tensor(0.8053)
350 tensor(0.8158)
400 tensor(0.8211)
450 tensor(0.8105)
500 tensor(0.8211)


In [0]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.2648, -0.2296, -0.2284, -0.0022, -0.0883, -0.1823, -0.1992, -0.1208],
         [ 0.0762,  0.0549, -0.1130,  0.3622, -0.3106, -0.1887,  0.3761,  0.0787],
         [ 0.3290, -0.2066, -0.2682, -0.0020, -0.1595, -0.2266, -0.1912,  0.3262],
         [-0.3164,  0.3207, -0.1899, -0.3152,  0.2069,  0.0322, -0.0067,  0.0748],
         [ 0.1319, -0.2061,  0.1027, -0.1297, -0.4038,  0.0167, -0.2884,  0.1112],
         [ 0.0202, -0.2644, -0.0765, -0.1151, -0.0815, -0.2204,  0.2370,  0.1984],
         [-0.2250, -0.1903, -0.3792,  0.3180,  0.3390, -0.0656, -0.0207, -0.0893],
         [ 0.0228, -0.1490,  0.2443,  0.1154,  0.1085, -0.2102, -0.4178,  0.1029],
         [ 0.2001,  0.3581, -0.2093, -0.0849, -0.1632, -0.0945, -0.2397,  0.0994],
         [-0.1126, -0.0832, -0.3842, -0.1975, -0.0985, -0.2712, -0.2656, -0.1384],
         [-0.1042, -0.3998,  0.0087, -0.1237, -0.2470, -0.2739,  0.0463,  0.1692],
         [ 0.3339,  0.0103, -0.1999,  0.0058,  0.2897,  0.1250, 